In [386]:
import numpy as np 
import IPython
from scipy.io.wavfile import read
import matplotlib.pyplot as plt
from scipy.fftpack import fft

# Initialize parameters 
n_fft = 4096     # fft length
fs = 44100       # sampling rate
n_hop = 256      # window overlap 256 - 1024 
n_templates = 30 # number of notes in library 
template_labels = np.array(["E2", "F2", "F#2", "G2", "G#2", "A3", "A#3", "B3", "C3", "C#3", "D3", "D#3", 
                   "E3", "F3", "F#3", "G3", "G#3", "A4", "A#4", "B4", "C4", "C#4", "D4", "D#4", 
                   "E4", "F4", "F#4", "G4", "G#4", "A5"])


In [1]:
# Writing NMF for polyphonic music analysis 

# Loads GuitarStrings.wav file from 02/26 and returns array of sustained data in training and testing array 
def load_data(snr):
    n_recordings = 59
    record_time = 4               # 4 seconds
    begin_sustain_time = 1
    end_sustain_time = 2

    noise_audio = "../data/noised_" + str(snr) + "db.wav"
    rate, arr = read("../data/GuitarStrings.wav")
    rate_noise, arr_noise = read(noise_audio)##
    arr_0 = arr[0:n_recordings*record_time*rate,0]
    arr_noise = arr_noise[0:n_recordings*record_time*rate]##
    
    begin_sustain = rate * begin_sustain_time
    end_sustain = rate * end_sustain_time

    print(end_sustain-begin_sustain)

    recordings = arr_0.reshape(n_recordings, record_time*rate)
    recordings_noise = arr_noise.reshape(n_recordings, record_time*rate)##
    
    sustains = recordings[:, begin_sustain:end_sustain]
    sustains_noise = recordings_noise[:, begin_sustain:end_sustain]##
    
    training = np.zeros((30, sustains.shape[1]))
    noised_1 = np.zeros((30, sustains.shape[1]))##
    training[0,:] = sustains[0,:]
    noised_1[0,:] = sustains_noise[0,:]##
    testing = np.zeros((30, sustains.shape[1]))
    noised_2 = np.zeros((30, sustains.shape[1]))##
    testing[0,:] = sustains[0,:]
    noised_2[0,:] = sustains_noise[0,:]##
    for i in range(1, training.shape[0]):
        training[i,:] = sustains[2*i-1,:]
        testing[i,:] = sustains[2*i, :]
        noised_1[i,:] = sustains_noise[2*i-1,:]##
        noised_2[i,:] = sustains_noise[2*i, :]##
        
    return training, testing, noised_1, noised_2
        
# Read Template data
def get_W(training):

    window_size = n_fft
    fft_iterations = np.floor((end_sustain - begin_sustain) / window_size)
    training = training[:, 0:fft_iterations*window_size]

    training = training.reshape(training.shape[0], fft_iterations, window_size)
    ffts = np.log(np.abs(fft(training, n = window_size, axis = 2)[:,:,0:n_fft/2]))
   
    # W dimension: number of templates x fft length
    W = np.mean(ffts, axis=1)[:]  
#     W = W[:, 0:window_size]
    W = W.T
    
    print (np.shape(W))
    index = np.argmax(W[0,0:1000])
    freqArray = np.arange(0, (n_fft/2), 1.0) * (rate*1.0/n_fft)
    print(freqArray[index])

    return W


In [393]:
# beta-divergence cost function 
# @args
# v - input sound vector 
# W - template library 
# h - activation vector 

# calculate cost by interpolating between square euclidian and 
def beta_divergence (v, W, h): 
    estimate = np.dot(W,h)
    # half euclidean distance (beta = 2)
    half_euc = 0.5*np.sum((v - estimate)**2)
    # kullback-liebler distance (beta = 1)
    k_liebler = np.sum(v*np.log(v/estimate) + v - estimate)
    cost = 0.5*(k_liebler + half_euc)

    return cost
    
    
    
# update activation matrix, h is updated in place
def update_estimate (v, W, h, beta):
    estimate = np.dot(W,h)
    v_matrix = np.outer(v, np.ones(h.shape[0]))
    numerator = np.dot( (W * v_matrix).T, np.power(estimate, (beta - 2)))
    denominator = np.dot( W.T, estimate**(beta-1) )
    h_new = h*(numerator/denominator)

    
    return h_new

    

# Iteratively update h until cost < absolute threshold and 
def iterate (cost_threshold, activation_threshold, v, W, h, beta, max_iter):
    i = 0
    cost = float('inf')
    prev_cost = -float('inf')

    while np.abs(prev_cost-cost) > cost_threshold and i < max_iter:
        prev_cost = cost
        cost = beta_divergence(v,W,h)
        
        h = update_estimate(v,W,h,beta)
        i += 1

    return h
                       

In [452]:
# Handling input data for test (not real time)

def test(window_size, cost_threshold, activation_threshold, v, W, h, beta):
    max_iter = 50000
    # Loop for overlapping fft
    result = np.zeros((30))
    num_iter = 0
    for i in range (0, len(v), n_hop):
        num_iter += 1
        h = np.ones(n_templates)
        h /= 10

        #Perform fft on window 
        if i + window_size > len(v):
            time_sequence = v[i:len(v)]
        else: 
            time_sequence = v[i: i + window_size]
        
        ffts = fft(time_sequence, 4096)
        fft_window = np.log(abs(ffts[0:n_fft/2]))
        #Iterate to find notes 
        h = iterate(cost_threshold, activation_threshold, fft_window, W, h, beta, max_iter)
        result += h
        
    result /= num_iter
    notes = template_labels[result > activation_threshold]
    return notes, result
        
        
        

In [468]:
snr = 10
train_data, test_data, noised_1, noised_2 = load_data(snr)
h = np.random.normal(0,1,n_templates)
W = get_W(train_data)
for i in range(0,30):
    notes, h_result = test( n_fft,  0.008, 0.05, noised_1[i,0:n_fft/2], W, h, 2)
    print("i: ", i, "output: ", np.argmax(h_result))
#     print(template_labels[np.argmax(h_result)])

44100
(2048, 30)
118.432617188


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel_launcher.py:47: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel_launcher.py:20: VisibleDeprecationWarn

i:  0 output:  7
i:  1 output:  7
i:  2 output:  5
i:  3 output:  7
i:  4 output:  7
i:  5 output:  5
i:  6 output:  6
i:  7 output:  7
i:  8 output:  7
i:  9 output:  7
i:  10 output:  10
i:  11 output:  7
i:  12 output:  7
i:  13 output:  7
i:  14 output:  6
i:  15 output:  15
i:  16 output:  16
i:  17 output:  7
i:  18 output:  18
i:  19 output:  19
i:  20 output:  7
i:  21 output:  5
i:  22 output:  22
i:  23 output:  5
i:  24 output:  24
i:  25 output:  5
i:  26 output:  7
i:  27 output:  7
i:  28 output:  16
i:  29 output:  29


In [417]:
notes


array([], 
      dtype='<U3')

In [448]:
print(np.argmax(h_result))
template_labels[np.argmax(h_result)]


7


'B3'